In [2]:
import pandas as pd
import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime
import json

## CDI

In [ ]:
df = pd.read_excel("datalake_files/DCF - RP'22 8&04 - Valores.xlsx",sheet_name = 'Budget',header = 5)
df = df.loc[~df['Unnamed: 1'].isnull()].drop('Unnamed: 0',axis = 1).set_index('Unnamed: 1').T
df.columns.name = None
df = df[list(map(lambda x: type(x) == datetime.datetime,df.index.values))]
serie = df['Cash Accumulated Available'] * 100

In [ ]:
def composed_interest(days,values):
    results = [values[0] + 1]
    for day,valor in zip(days[1:],values[1:]):
        if day == 0:
            results.append(1)
        else:
            results.append(results[-1] * valor + 1)
    return np.array(results) - 1

In [ ]:
arquivo = 'datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx'
dfs = []
messages = []
for i in range(1,11):
    try:
        df = pd.read_excel(arquivo,sheet_name = f'{i}ª Emissão CDI',header = 4)
        df = df[df['Filtro'] == 'X'][['Data','Last  CDI','Days','Interest (Month)','Principal Balance']].dropna()
        df['Data'] = pd.to_datetime(df['Data'],format = '%d%m%Y')
        if df['Data'].iloc[0] < datetime.datetime.now():
            df['Last  CDI'] = df['Last  CDI'] / 100
            spread = pd.read_excel(arquivo,sheet_name = '1ª Emissão CDI').iloc[2,9] / 100
            df['Spread'] = spread
            df['Taxa Diária Estimada'] = ((df['Last  CDI'] + spread + 1) ** (1/252)) - 1
            df['Days Dif'] = df['Days'].diff().fillna(df['Days'].iloc[0]).apply(lambda x: 0 if x < 2 else x)
            df['Taxa Estimada Período'] = ((1 + df['Taxa Diária Estimada']) ** df['Days Dif']) - 1
            df['Taxa Acumulada Estimada'] = composed_interest(df['Days Dif'].values,df['Taxa Estimada Período'].values)
            df['Juros Estimados'] = df['Principal Balance'] * df['Taxa Acumulada Estimada']
            df['Despesa Estimada'] = df['Juros Estimados'].cumsum()
            df['Despesa Real'] = (df['Interest (Month)'] * df['Days Dif'].apply(lambda x: 0 if x == 0 else 1)).cumsum()
            df['Erro'] = 100 * (df['Despesa Real'] - df['Despesa Estimada']) / df['Despesa Real']
            df['Período'] = df['Data'].apply(lambda x: str(x.year) + '-' + str(x.month))
            dfs.append(df)
            messages.append(f'{i}ª Emissão CDI: Sucesso')
        else:
            messages.append(f'{i}ª Emissão CDI: Não Realizado')
    except ValueError as e:
        print(e)
        messages.append(f'{i}ª Emissão CDI: ' + str(e))

## IPCA

In [ ]:
arquivo = 'datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx'
i = 1
positions = (15,14)
emissao = i + 1
df = pd.read_excel(arquivo,sheet_name = f'{emissao}ª Emissão IPCA',header = 6)
df = df[df['Filtro'] == 'X'][['Date','IPCA','Interest','Monetary Variation',' Principal Balance']]
df['Date'] = pd.to_datetime(df['Date'],format = '%d%m%Y')
raw = pd.read_excel(arquivo,sheet_name = f'{emissao}ª Emissão IPCA')
total = raw.iloc[0,positions[i]]
juros = raw.iloc[4,positions[i]]
del(raw)
df['Taxa'] = juros
df['Capital'] = total
juros_semestral = ((1 + (juros / 100)) ** (1/2)) - 1
df['Days Dif'] = df['Date'].diff().apply(lambda x: x.days).fillna(30)
df = df[df['Days Dif'] <= 1]
df['Monetary Variation'] = df['Monetary Variation'].fillna(0)
ipca_base = (df['Capital'].iloc[0] / df[' Principal Balance'].iloc[0]) * df['IPCA'].iloc[0]
subtract = 0
for i in df.index:
    if df.loc[i,'Days Dif'] == 1:
        subtract += df.loc[i,'Interest']
    df.loc[i,'Capital'] -= subtract
df = df[df['Days Dif'] != 1]
df['Despesa Acumulada'] = (df['Interest'] + df['Monetary Variation']).cumsum()
df['Capital Dif'] = df['Capital'].diff().fillna(0).cumsum()
df['Principal Balance Calculado'] = (total * df['IPCA'] / ipca_base) + (df['Capital Dif'] * df['IPCA'] / ipca_base)
df['Interest Calculado'] = df['Principal Balance Calculado'] * juros_semestral
faltante = (total + sum(df['Capital'].diff().dropna())) * (-1)
df['Monetary Variation Calculado'] = df['Capital'].diff().apply(lambda x: None if x == 0 else x).fillna(method = 'bfill').fillna(faltante) * (-1) * ((df['IPCA'] / ipca_base) - 1) * df['Monetary Variation'].apply(lambda x: 1 if x > 0 else 0)
df['Despesa Acumulada Calculado'] = (df['Interest Calculado'] + df['Monetary Variation Calculado']).cumsum()
df['Erro'] = 100 * (df['Despesa Acumulada'] - df['Despesa Acumulada Calculado']) / df['Despesa Acumulada']

In [55]:
df = pd.read_excel("datalake_files/DCF - RP'22 8&04 - Valores.xlsx",sheet_name = 'Budget',header = 5)
df = df.loc[~df['Unnamed: 1'].isnull()].drop('Unnamed: 0',axis = 1).set_index('Unnamed: 1').T
df.columns.name = None
df = df[list(map(lambda x: type(x) == datetime.datetime,df.index.values))]
serie = df['Operational Costs'] * 100

## Câmbio

In [ ]:
df = pd.read_excel('datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx',sheet_name = 'Daily Calculation prop2019',header = 2)
df = df[['Date','USD','Repayment']].dropna()
df = df.set_index('Date')
df = df[datetime.datetime.today():]
df['Period'] = df.index.to_period('m')
df['Reais_Acumulado'] = (df['USD'] * df['Repayment']).cumsum()

## Trading

In [4]:
def ultimo_reajuste(row):
    mes_de_reajuste = row['ReajusteDataBase'].month
    mes_atual = row['Competencia'].month
    ano_atual = row['Competencia'].year
    if mes_atual >= mes_de_reajuste:
        data = pd.Period(f'{ano_atual}-{mes_de_reajuste}')
    else:
        data = pd.Period(f'{ano_atual - 1}-{mes_de_reajuste}')
    return max(data,row['ReajusteDataBase'])

def absolute(serie,base):
    valor_atual = base
    yield valor_atual
    for valor in serie[:-1]:
        valor = valor / 100
        valor_atual += (valor_atual * valor)
        yield valor_atual

In [5]:
lca_anual = pd.read_excel('datalake_files/Projecoes_LCA.xlsx','Base_Anual',header = 11).dropna().set_index('Unnamed: 0').T['IPCA - IBGE (% a.a.)']
lca_anual = pd.Series(lca_anual.values,index = list(map(lambda x: int(str(x)[:4]),lca_anual.index))).loc[2000:]
lca_anual = pd.Series([valor for valor in absolute(lca_anual.values,1598.41)],index = lca_anual.index,name = 'ipca')
df_anual = pd.DataFrame(index = pd.period_range(start = '{}-01-01'.format(lca_anual.index[0]),end = '{}-01-01'.format(lca_anual.index[-1]),freq = 'M'))
df_anual['ano'] = df_anual.index.year
df_anual = df_anual.join(lca_anual,on = 'ano')
df_anual['lca_anual'] = list(df_anual['ipca'].rolling(12).mean())[12:] + ([None] * 12)
lca_anual = df_anual['lca_anual']
lca_mensal = pd.read_excel('datalake_files/Projecoes_LCA.xlsx','Base_Mensal',header = 8).drop([0,1,2]).set_index('Período')['IPCA']
first_year = lca_mensal.index.year[0]
lca_mensal = pd.Series([valor for valor in absolute(lca_mensal.values,lca_anual[f'{first_year}-01'])],index = lca_mensal.index.to_period('M'),name = 'lca_mensal')
arquivo = 'datalake_files/2021_01_05_12_14_56_wbc.json'
df = pd.DataFrame.from_records(json.loads(open(arquivo,'r',encoding = 'utf-8').read())['Values'])
df = df[['ReajusteDataBase','Competencia','DataParcela1','QuantSolicitada','Valor','ValorReajustado']].dropna()
df['ReajusteDataBase'],df['Competencia'],df['DataParcela1'] = pd.to_datetime(df['ReajusteDataBase']).apply(lambda x: pd.Period(x,'M')),pd.to_datetime(df['Competencia']).apply(lambda x: pd.Period(x,'M')),pd.to_datetime(df['DataParcela1']).apply(lambda x: pd.Period(x,'M'))
df['UltimoReajuste'] = df.apply(ultimo_reajuste,axis = 1)
df['Valor'] = df['Valor'].apply(float)
df = df.join(lca_mensal,on = 'UltimoReajuste')
df = df.join(lca_mensal,on = 'ReajusteDataBase',rsuffix = '_base')
df = df.join(lca_anual,on = 'UltimoReajuste')
df = df.join(lca_anual,on = 'ReajusteDataBase',rsuffix = '_base')
df['ipca'] = df['ipca'].fillna(df['ipca_ma'])
df['ipca_base'] = df['ipca_base'].fillna(df['ipca_ma_base'])
del(df['ipca_ma'])
del(df['ipca_ma_base'])
df['ValorReajustadoCalculado'] = (df['ipca'] / df['ipca_base']) * df['Valor']
df['Total'] = df['QuantSolicitada'] * df['ValorReajustado']

In [7]:
data_inicio,data_fim = pd.Period('2023-01'),pd.Period('2027-12')
df = df[(df['Competencia'] >= data_inicio) & (df['Competencia'] <= data_fim)]